## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ **Determine business objectives**:   Construir um detector de fake news.
+ **Assess situation**:  A detecção de fake news já faz parte do subconsciente do usuário diário da internet. Felizmente podemos traduzir nossa intuição por trás de um título duvidoso e de palavras no imperativo para termos de Machine Learing. 
+ **Determine data mining goals** :  Buscar por sites de notícias que analisam veracidade de notícias, extraindo URLs, manchetes, subtítulos e outras informações pertinentes.
+ **Produce project plan** :  
Avaliar fontes de coleta de dados.
Selecionar critério para confiabilidade de um site.
Coletar e limpar os dados.
Construir e examinar modelo Tfidf.
Averiguar os resultados e reavaliar o modelo.
Revisar o projeto e realizar o deploy.





## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

### Metas!!!

+ Organizar categorias de notícias e os veículos de transmissão
+ Encontrar pessoas/instiuições/temas recorrentes
+ Atentar para caracteres Unicode
+ Determinar filtros (o que vamos armazenar do texto fonte).

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
from collections import Counter
from collections import defaultdict

import nltk


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import requests
import urllib
from bs4 import BeautifulSoup as bs

In [2]:
datapath = "../data"

class_name_error = "erro-wrapper"
class_name_label = "etiqueta"
url = 'https://piaui.folha.uol.com.br/lupa/tag/verificamos/'





In [3]:
url = 'https://piaui.folha.uol.com.br/lupa/tag/verificamos/'
content = requests.get(url)
print(content.status_code)

200


In [4]:
def check_block(bloco):   
    try:
        title = bloco.find_element_by_class_name("bloco-title")
    except:
        return False
    
    verificamos = re.search("\#Verificamos",title.text)
    
    if verificamos is None:
        return False
    else:
        return True

def extract_link(bloco):
    
    return bloco.find_element_by_tag_name("a").get_attribute("href")


def extract_title(texto):
    
    regex = r'Confira.*\n.*(?<=“)(.*)(?=”)'
    
    match = re.search(regex, texto)
    
    if match is None:
        return None
    
    return match.group(1)



def page_out_of_bounds(driver):
    
    penguin = driver.find_elements_by_class_name(class_name_error)
    
    if len(penguin) == 0:
        return False
    else:
        return True
    
def extract_date_and_type(bloco):
    
    info = bloco.find_element_by_class_name('bloco-meta').text
    regex = r'(\d{2}\.\d{2}\.\d{4}).\|.*\|.(.*)'
    
    match = re.search(regex,info)
    date = match.group(1)
    tipo = match.group(2)
    
    return (date,tipo)
    
    

# Prepare to scrape!

In [5]:
options = webdriver.chrome.options.Options()
options.headless = True

driver  = webdriver.Chrome(ChromeDriverManager().install(), options = options)
crawler = webdriver.Chrome(ChromeDriverManager().install(), options = options)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_linux64.zip
Driver has been saved in cache [/home/isaque/.wdm/drivers/chromedriver/linux64/93.0.4577.63]


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/home/isaque/.wdm/drivers/chromedriver/linux64/93.0.4577.63/chromedriver] found in cache


In [57]:
N = 1

while True:
    
    page = url + 'page/' + str(N)
    
    driver.get(page)
    
    if page_out_of_bounds(driver):
        break
    blocos = driver.find_elements_by_class_name('bloco')
    
    for bloco in blocos:
        
        if check_block(bloco):
            
        
            link = extract_link(bloco)
            date_tipo = extract_date_and_type(bloco)
            
            crawler.get(link)
            label = crawler.find_element_by_class_name(class_name_label).text
            titulo = crawler.find_element_by_class_name("post-inner")
            string = titulo.text
            print(string)
            noticia_original = extract_title(string)
            print()
            print(label)
            print()
            print(noticia_original)
            print()
            print(date_tipo[0])
            print()
            print(date_tipo[1])
            print()
            print(20*'-')
            print()
        else:
            continue
    N += 1


Circula pelas redes sociais uma foto que mostra dezenas de carros parados em uma rodovia. De acordo com a legenda, a imagem retrataria uma suposta manifestação na Alemanha em que a população teria deixado 1 milhão de carros nas ruas para obstruir a circulação e protestar contra o preço dos combustíveis. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“Na Alemanha o governo subiu preço de combustível. Em apenas uma hora de tempo as pessoas abandoram seus carros nas ruas e avenidas e foram a pé para casa. Mais de um milhão de carros abandonados. Tiveram que baixar o preço. Quando o povo é inteligente os corruptos não conseguem concretizar seus objetivos”
Legenda de imagem publicada no Facebook que, até 13h do dia 15 de setembro de 2021, tinha mais de 2,8 mil compartilhamentos
FALSO
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
A informação analisada

Circula no WhatsApp a informação de que um sobrinho do ex-presidente Luiz Inácio Lula da Silva (PT) foi preso no município de Canarana, no Mato Grosso, com R$ 6 milhões. De acordo com a publicação — um print de um post no Facebook —, a delegacia da cidade estaria cercada porque a população ameaçou invadir o local. O post é acompanhado de duas fotos, que teriam sido tiradas nessa delegacia. A primeira delas mostra dois homens (ao centro) escoltados por policiais. A segunda imagem, logo abaixo, mostra o que parece ser uma sala da Polícia Civil onde vários policiais posam ao redor de uma mesa com cédulas de dinheiro. Por WhatsApp, leitores da Lupa sugeriram que esse conteúdo fosse analisado. Confira a seguir o trabalho de verificação:
“Manda pra todo mundo que connhecer. Sobrinho do Lula foi preso em Canarana/MT com aproximadamente 6 milhões de reais e disse para a Polícia que era do tio Lula e foi para comprar uma fazenda.
A delegacia está cercada à 200 mts pq a população ameaça invadir 

Circula no WhatsApp o vídeo de uma entrevista com a médica Maria Emilia Gadelha Serra. Na gravação, de mais de 4 minutos, ela discorre sobre vários supostos problemas relacionados às vacinas contra a Covid-19. A médica começa afirmando que a maioria dos casos graves da doença hoje no Brasil ocorre em pessoas que já foram vacinadas. 
Sobre a CoronaVac, Serra diz, entre outros pontos, que o Instituto Butantan, que desenvolveu a vacina no Brasil em parceria com o laboratório chinês Sinovac, não informa a quantidade de alumínio na fórmula. Ela também afirma que, durante os estudos dessa vacina, foi usado um tipo de placebo com 500 microgramas de alumínio na fórmula, o que seria tóxico. Ainda sobre o alumínio, a profissional sugere que esse metal está associado ao Alzheimer. 
Ao falar dos imunizantes da Janssen, da AstraZeneca e da Pfizer/ BionTech, a médica sugere que essas fórmulas transformam o organismo das pessoas vacinadas em “fábricas de proteína spike”, a proteína do novo coronavíru

Circula pelas redes sociais um vídeo em que o cantor Eduardo Costa afirma que 80% das urnas no Brasil são fraudadas. Ele diz ainda que, se o presidente Jair Bolsonaro (sem partido) não vencer as próximas eleições, vai acontecer uma guerra civil no país. “Vocês estão roubando o povo na cara dura”, afirmou. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“80% das urnas no Brasil estão fraudadas”
Vídeo publicado no Facebook que, até às 19h horas do dia 8 de setembro foi visualizado 2,4 mil vezes
FALSO
A informação analisada pela Lupa é falsa. Segundo o Tribunal Superior Eleitoral (TSE), nenhuma fraude foi comprovada em 24 anos de uso do equipamento. Além disso, a Polícia Federal não encontrou registros de investigações sobre fraudes que comprometessem o sistema desde que começou a ser usado. O vídeo de Eduardo Costa foi publicado em outubro de 2018, quando o cantor fez a

Circula pelas redes sociais a informação de que a Transparência Internacional no Brasil, organização global de combate à corrupção, recentemente publicou uma nota na qual repudiava o inquérito das fake news, que tem como relator o ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF). Esse conteúdo foi noticiado como sendo “urgente” e atual durante cobertura dos atos do 7 de setembro pelo canal Terça Livre no YouTube. No final da transmissão, um dos apresentadores afirmou se tratar de uma informação recente. Ele leu alguns trechos do documento, como a parte que afirmava que “o inquérito das fake news abre precedente para grave retrocesso no império da lei e defesa de liberdades”. O apresentador também reproduziu na tela do vídeo uma imagem da nota publicada no site da organização.
Essa informação foi compartilhada por Filipe G. Martins, assessor especial para assuntos internacionais do presidente Jair Bolsonaro (sem partido), em seu perfil no Twitter. Além dele, o ex-assessor

...

Circula pelas redes sociais que a ex-presidente Dilma Rousseff (PT) matou um soldado nos anos 1960. A publicação, que já circulou em anos anteriores, diz que a ex-presidente da república jogou uma granada no portão do quartel general do exército e que esse atentado culminou com a morte do soldado Mário Kozel Filho. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“DA SÉRIE: “COISAS QUE VOCÊ NUNCA APRENDEU NOS LIVROS DE HISTÓRIA DO MEC.
HOJE FAZ 52 ANOS QUE A EX PRESIDENTE DILMA ROUSSEFF MATOU O SOLDADO MARIO KOZEL FILHO, JOGANDO UMA GRANADA NO PORTÃO DO QUARTEL GENERAL DO EXÉRCITO. SIM, TIVEMOS UMA TERRORISTA ASSASSINA COMO PRESIDENTE!”
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
Texto em post que circula no Facebook que, até as 14h de 25 de maio de 2021, tinha mais de 380 compartilhamentos
FALSO
A informação analisada pela Lupa é falsa. A ex-pre

Circula pelo WhatsApp um texto que diz que pessoas imunizadas com a vacina da Pfizer/BioNTech podem transmitir o vírus da covid-19 por meio de inalação ou contato com a pele. O artigo menciona ainda o risco de grávidas sofrerem abortos espontâneos e outros problemas reprodutivos ao tomarem a vacina da farmacêutica norte-americana. Por WhatsApp, leitores da Lupa sugeriram que esse conteúdo fosse analisado. Confira a seguir o trabalho de verificação:
“Pfizer admite em seu próprio documento de mRNA que os não vacinados podem ser expostos às proteínas spike da vacina por inalação ou contato com a pele”
Título de texto publicado no site Anonymous Incision, e que circula em grupos de WhatsApp
FALSO
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
 A informação analisada pela Lupa é falsa. O texto cita como “fonte” o relatório final da pesquisa desenvolvida pela Pfizer para o desenvolvimento de sua vacina contra a Covid-19, publicado em dezembro de 2020. Contudo, o texto tra

Circula pelas redes sociais que o presidente dos Estados Unidos, Joe Biden, retomou a construção do muro na fronteira com o México. Essa obra tinha sido iniciada pelo antecessor, o republicano Donald Trump. Por meio do projeto de verificação de notícias, usuários do Facebook solicitaram que esse material fosse analisado. Confira a seguir o trabalho de verificação da Lupa:
“BIDEN É RACISTA TAMBÉM?
APÓS CRISE MIGRATÓRIA, BIDEN RETOMA CONSTRUÇÃO DO MURO DE TRUMP”
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
Texto em post circula no Facebook que, até as 13h30 de 21 de maio de 2021, tinha sido compartilhado 112 vezes
FALSO
A informação analisada pela Lupa é falsa. Embora a questão migratória tenha se tornado a primeira crise do atual governo norte-americano, não é verdade que Joe Biden retomou a construção do muro fronteiriço entre os Estados Unidos e o México, conhecido como “Trump Wall” (Muro de Trump). Em 20 de janeiro, primeiro dia como presidente, o democrata assi

Circula por grupos de WhatsApp um vídeo que mostra a explosão de um carro-bomba e, na sequência, um grupo de pessoas que finge ter sido atingido pela detonação. Outro grupo aparece para socorrer as supostas vítimas. De acordo com a publicação, essa encenação foi feita pelo Hamas, grupo de militantes islâmicos da Palestina atualmente envolvido em conflitos com Israel. O narrador sugere que a explosão forjada tem o objetivo de incriminar Israel. Por WhatsApp, leitores da Lupa sugeriram que esse conteúdo fosse analisado. Confira a seguir o trabalho de verificação:
“A industria das Fake News do Hamas
Bom dia seguidores da página Viva Israel, estou trazendo para vocês mais um vídeo da indústria de cinema. É praticamente uma indústria de cinema do grupo Hamas. O que a gente tá vendo aqui é um carro que será explodido como se Israel tivesse atingido ele, esse carro com mísseis. (…) Logo em seguida começam a entrar os figurantes, que vão se jogar no chão, como se tivessem sido as pessoas que s

KeyboardInterrupt: 

In [39]:
print("“" == "”")

False


## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 